In [ ]:
import pandas as pd
import math
import os
from tqdm.notebook import tqdm
import networkx as nx
import numpy as np
import scipy.sparse as sp
# from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from deep_translator import GoogleTranslator

In [ ]:
def summarize_textrank(sentences):
    # sentence_tokenizer = PunktSentenceTokenizer()
    # sentences = sentence_tokenizer.tokenize(document)

    bow_matrix = CountVectorizer().fit_transform(sentences)
    normalized = TfidfTransformer().fit_transform(bow_matrix)

    similarity_graph = normalized * normalized.T
    sparse_matrix = sp.csr_matrix(similarity_graph)
    nx_graph = nx.Graph(sparse_matrix)
    scores = nx.pagerank(nx_graph)
    sentence_array = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    
    sentence_array = np.asarray(sentence_array)
    
    fmax = float(sentence_array[0][0])
    fmin = float(sentence_array[len(sentence_array) - 1][0])
    
    temp_array = []
    for i in range(0, len(sentence_array)):
        if fmax - fmin == 0:
            temp_array.append(0)
        else:
            temp_array.append((float(sentence_array[i][0]) - fmin) / (fmax - fmin))

    threshold = (sum(temp_array) / len(temp_array)) + 0.2
    
    sentence_list = []

    for i in range(0, len(temp_array)):
        if temp_array[i] > threshold:
            sentence_list.append(sentence_array[i][1])

    seq_list = []
    for sentence in sentences:
        if sentence in sentence_list:
            seq_list.append(sentence)
    return seq_list

In [ ]:
input_path="../data/"
output_path="../results/summary/"
os.makedirs(output_path, exist_ok=True)
if os.path.exists(output_path) == False:
    os.mkdir(output_path)

data = pd.read_csv(f'{input_path}data.csv')
sentences = data['sentences'].apply(eval)

summary=[]
for i in tqdm(range(len(sentences))):
    summary.append(summarize_textrank( sentences[i]) ) 

data['summary'] = summary

data.to_csv(f'{output_path}textRank.csv', index=False)